In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
import torch.utils.data

torch.manual_seed(3407)

df = pd.read_csv("./ESC-50/meta/esc50.csv")


# df.head()
# 选择每个类别的前 60% 数据
def select(group, start_percent, end_percent):
    start = int(len(group) * start_percent)  # 计算 60% 的位置
    end = int(len(group) * end_percent)  # 计算 80% 的位置
    return group.iloc[start:end]


# df.groupby("category")
sampled_df = df.groupby('category').apply(lambda x: select(x,0,0.6))

# 重置索引
sampled_df = sampled_df.reset_index(drop=True)

FileNotFoundError: [Errno 2] No such file or directory: './ESC-50/meta/esc50.csv'

In [4]:
categories = df["category"].nunique()
print(f"有{categories}种取值")

NameError: name 'df' is not defined

接下来，准备数据集

In [ ]:
import torchaudio
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from src import label_to_index
from torch.utils.data import Dataset, DataLoader


class SoundDataset(Dataset):
    def __init__(self):
        self.transformation = torchaudio.transforms.Spectrogram()
        self.target_sample_rate = 16_000

    def __len__(self):
        return len(df)

    def __getitem__(self, item):
        row = df.take([item], axis=0)

        file_path = "./ESC-50/audio/" + row.filename.values[0]
        waveform, sample_rate = torchaudio.load(file_path)
        if sample_rate != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=self.target_sample_rate)
            waveform = resampler(waveform)

        waveform = self.transformation(waveform)
        # mel_spectrogram = MelSpectrogram(sample_rate)(waveform)
        # mel_spectrogram_db = AmplitudeToDB()(mel_spectrogram)

        return waveform, label_to_index(row.category.values[0])


dataset = SoundDataset()
dataloader = DataLoader(dataset, shuffle=True, batch_size=128)

接下来编写模型

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(160_000, 64)
        self.fc2 = nn.Linear(64, categories)  # 假设有10个类别

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = SimpleCNN()
model.to("mps")


In [16]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().to("mps")
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):  # 迭代多个周期
    for i, (inputs, labels) in enumerate(dataloader):
        labels = labels.to("mps")
        inputs = inputs.to("mps")
        # 梯度清零
        optimizer.zero_grad()

        # 前向 + 反向 + 优化
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f'Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {loss.item()}')


NameError: name 'model' is not defined